# Timing Data

In [ ]:
from time_solvers import timed_trials, plot_solver_run_times
import numpy as np

In [ ]:
from gillespy2 import StochKitSolver, NumPySSASolver, OptimizedSSASolver, SSACSolver
from gillespy2.example_models import Example, Trichloroethylene, MichaelisMenten, Schlogl
modelList = [Example(), Trichloroethylene(), MichaelisMenten(), Schlogl()]
solverList = [ StochKitSolver, NumPySSASolver, OptimizedSSASolver, SSACSolver]

In [ ]:
trajectories = [2**i for i in range(11)]
timing_data = timed_trials(modelList, solverList, trajectories, number_trials=30,output_file='timing_data.pkl')

In [ ]:
#Ideally, we can run the timed tests once and save the results for later in a .pkl file
import pickle
with open('timing_data.pkl', 'rb') as f:
    timing_data = pickle.load(f)
    
%matplotlib inline
styles = {
    'StochKitSolver' : 'r*-',
    'SSACSolver' : 'g*-',
    'CythonSSASolver' : 'b*-',
    'NumPySSASolver' : 'k*-'    
}

In [ ]:
plot_solver_run_times(timing_data, line_styles=styles)

In [ ]:
plot_solver_run_times(timing_data, line_styles=styles, ylabel='Log(seconds)', transformation=np.log)

In [ ]:
plot_solver_run_times(timing_data, line_styles=styles, ylabel='Relative Speed to NumPySolver', baseline_solver_name=NumPySSASolver.name)